In [1]:
# https://spacy.io/usage/linguistic-features
# https://www.dataquest.io/blog/tutorial-text-classification-in-python-using-spacy/
# https://nlpforhackers.io/complete-guide-to-spacy/
import spacy
import pandas as pd
#for visualization of Entity detection importing displacy from spacy
from spacy import displacy
import nltk
# https://www.analyticsvidhya.com/blog/2019/02/stanfordnlp-nlp-library-python
# sentence tokenisation Load English tokenizer, tagger, parser, NER and word vectors
# nlp = English()

import en_core_web_sm
# import en_core_web_lg
from spacy.pipeline import Sentencizer
sentencizer = Sentencizer()
# Create the pipeline 'sentencizer' component
# sbd = nlp.create_pipe('sentencizer')
# Add the component to the pipeline
# nlp.add_pipe(sbd)

nlp = spacy.load("en_core_web_sm")
nlp.add_pipe(sentencizer, first=True)

# dependency
root = 'root'
direct_obj = 'dobj' 
nominal_subj = 'nsubj' 
prep = 'prep'
clausal_complement = ['ccomp', 'advcl']
compound = 'compound'
list_dep = ['compound', 'pobj', root, nominal_subj, direct_obj]
# preposition
verb = 'VERB'
noun = 'NOUN'
root = 'ROOT'
adp = 'ADP'
verb_noun = [verb, noun, 'PRON']



In [2]:
def read_playbook(filename, column):
    # read playbook or task name
    df_content = pd.read_csv(filename)
    column_content = df_content[column]
      # drop empty cell
    column_content = column_content.dropna()
    # drop duplicates cells
    column_content = column_content.drop_duplicates() 
    # print(playbook_name.head())
    # print('\n *********************************** \n')
    return column_content



In [3]:
def separate_sentences(sentences, nlp):
    # first separate the sentence in the playbok
    sentence_list = []
    for sentences in sentences.values:
        # "nlp" Object is used to create documents with linguistic annotations.
        doc = nlp(sentences)
        # create list of sentence tokens
        for sent in doc.sents: 
            sentence_list.append(sent.text.lower())
            # print(sent.text, '\n')
    print('Number of sentence: ', len(sentence_list))
    return sentence_list



In [4]:
# read playbook details
playbook_name = read_playbook('Playbook.csv', 'name')
# read task details
task_plan = read_playbook('Task_playbook.csv', 'description')
# read task name
task_name = read_playbook('Task_playbook.csv', 'name')
sentence_list = []
# first separate the sentence in the playbok
sentence_list = separate_sentences(task_plan, nlp)



Number of sentence:  1041


In [5]:
def calculate_api_param(token, token_comp):
    # token_comp = token.lemma_
    for child in token.children:
        if child.dep_== 'compound':
            new_str = str(child.lemma_ + '.' + token.lemma_)
            old_str = str(token.lemma_)
            token_comp = token_comp.replace(old_str, new_str)
            # print('compound:', token_comp)
            for token1 in child.children:
                if token1.dep_ == 'compound':
                    token_comp = calculate_api_param(child, token_comp)
    return token_comp



In [6]:
def calculate_compound(token, token_comp):
    # token_comp = token.lemma_
    for child in token.children:
        if child.dep_== 'compound' or child.dep_=='amod':
            # token_comp = str(child.lemma_ + '.' + token_comp)
            new_str = str(child.lemma_ + '.' + token.lemma_)
            token_comp = token_comp.replace(str(token.lemma_), new_str)
            # print('compound:', token_comp)
            for token1 in child.children:
                if token1.dep_ == 'compound':
                    token_comp = calculate_compound(child, token_comp)
    return token_comp


In [77]:
# identify First API - the task that need to be done
def generate_api(doc):   
    first_api = second_api = third_api = first_api_compound = second_api_compound = third_api_compound = ""
    for token in doc:
        f = s = t =v = 0.00
        token_dep = token.dep_
        token_pos = token.pos_    
        child = [i for i in token.children]
        # print('token_dep: ', token_dep, 'token_pos: ', token_pos)
        # rules for identifying first API
        if token_dep == root and token_pos in verb_noun: # rule 1    
            first_api = token.lemma_
            print('rule 1: ', first_api)
            f = 1
            for child in token.children:
                if child.dep_ == compound and child.pos_ in verb_noun: # identify the compound of root
                    first_api = child.lemma_
                    first_api_compound = calculate_compound(child, child.lemma_)
                    first_api_compound = first_api_compound.replace(child.lemma_, "")
                    second_api = token.lemma_
                    second_api_compound = calculate_compound(token, token.lemma_)
                    second_api_compound = second_api_compound.replace(token.lemma_, "")
                    print('rule 2.1: ', first_api, '.', second_api)
                    print('first_api_param', first_api_compound,' second_api_param ', second_api_compound)
                    f = s = 2.1
                elif child.dep_ in [nominal_subj, 'amod']  and child.pos_ in [verb_noun, 'ADJ']: # identify the subject of root
                    children_child = [i for i in child.children]
                    if(len(children_child) == 0):
                        first_api =  child.lemma_
                        first_api_compound = calculate_compound(child, child.lemma_)
                        first_api_compound = first_api_compound.replace(child.lemma_, "")
                        second_api = token.lemma_
                        second_api_compound = calculate_compound(token, token.lemma_)
                        second_api_compound = second_api_compound.replace(token.lemma_, "")
                        print('rule 2.2: ', first_api, '.', second_api)
                        print('first_api_param', first_api_compound,' second_api_param ', second_api_compound)
                        f = s = 2.2
                elif child.dep_ == 'dep'  and child.pos_ in verb_noun: # identify the subject of root
                    children_child = [i for i in child.children]
                    if(len(children_child) == 0):
                        third_api = second_api
                        third_api_compound = second_api_compound
                        second_api = first_api
                        second_api_compound = first_api_compound
                        first_api = child.lemma_ 
                        first_api_compound = calculate_compound(child, child.lemma_)
                        first_api_compound = first_api_compound.replace(child.lemma_, "")
                        print('rule 2.3: ', first_api, '.', second_api)
                        print('first_api_param', first_api_compound,' second_api_param ', second_api_compound,'third_api_param ', third_api_compound)
                        f = s = 2.3
                elif child.dep_ in clausal_complement and child.pos_ == verb:
                    for token1 in child.children:
                        if token1.dep_ == nominal_subj:
                            second_api = token1.lemma_
                            second_api_compound = calculate_compound(token1, token1.lemma_)
                            second_api_compound = second_api_compound.replace(token1.lemma_, "")
                            print('rule 2.5: ', first_api, '. ', second_api)
                            print('second_api_param', second_api_compound)
                            f = s = 2.5
                        elif token1.dep_== direct_obj:
                            third_api = token1.lemma_  
                            third_api_compound = calculate_compound(token1, token1.lemma_)
                            third_api_compound = third_api_compound.replace(token1.lemma_, "")
                            print('rule 3.5: ', first_api, '. ', second_api, '. ', third_api)
                            print('third_api_param', third_api_compound)
                            t = 3.5
                elif child.dep_ == direct_obj and child.pos_ in verb_noun:
                    if s == 0:
                        second_api = child.lemma_
                        second_api_compound = calculate_compound(child, child.lemma_)
                        second_api_compound = second_api_compound.replace(child.lemma_, "")
                        print('rule 2.4: ', first_api, '. ', second_api) # rule 2
                        print('second_api_param', second_api_compound)
                        s = 2.4
                        for token1 in child.children:
                            if token1.dep_ == 'prep' or 'adative' and token1.pos_ == 'ADP':
                                for token2 in token1.children:
                                    third_api = token2.lemma_
                                    t = 3.41
                                    print('rule 3.1: ', first_api, '. ', second_api, '.', third_api)
                                    third_api_compound = calculate_compound(token2, token2.lemma_)
                                    third_api_compound = third_api_compound.replace(token2.lemma_, "")
                                    print('third_api_param', third_api_compound)
                    else:
                        third_api = child.lemma_
                        third_api_compound = calculate_compound(child, child.lemma_)
                        third_api_compound = third_api_compound.replace(child.lemma_, "")
                        print('rule 3.4: ', first_api, '. ', second_api, '.', third_api) # rule 2
                        print('third_api_param', third_api_compound)
                        t = 3.4
                elif child.dep_ == 'prep' or 'adative' and child.pos_ == 'ADP':
                    for token1 in child.children:
                        if token1.dep_ == 'pobj' and token1.pos_ in verb_noun:
                            if s == 0:
                                second_api = token1.lemma_
                                second_api_compound = calculate_compound(token1, token1.lemma_)
                                print('rule 2.1: ', first_api, '. ', second_api)
                                s = 2.1
                                print('second_api_param', second_api_compound)
                            else:
                                third_api = token1.lemma_
                                t = 3.1
                                print('rule 3.1: ', first_api, '. ', second_api, '.', third_api)
                                third_api_compound = calculate_compound(token1, token1.lemma_)
                                third_api_compound = third_api_compound.replace(token1.lemma_, "")
                                print('third_api_param', third_api_compound)
                elif child.dep_ in ['pobj', 'advcl'] and child.pos_ in ['ADJ', 'NOUN', 'PRON']:
                    if t == 0:
                        third_api = child.lemma_
                        #print('pobject', third_param)
                        t = 3.6
                        print('rule 3.6: ', first_api, '. ', second_api, '.', third_api)
                        third_api_compound = calculate_compound(child, child.lemma_)
                        third_api_compound = third_api_compound.replace(child.lemma_, "")
                        print('third_api_param', third_api_compound)
                                
    return first_api, second_api, third_api

In [78]:
count = 0
for sentence in sentence_list:
    if(count < 210):
        count = count + 1
        continue
    elif count == 250:
        break
    count = count + 1
    print('\n', sentence, '\n')
    doc = nlp(sentence)
    first_api = second_api = third_api = first_api_feature = second_api_feature = third_api_feature = ""
    first_api, second_api, third_api = generate_api(doc)
    # generate_api_with_compound(doc)
    if third_api:
        if not second_api:
            print(first_api, '.', third_api)
        else:
            print(first_api, '.', second_api, '.', third_api)
    else:
        print(first_api, '.', second_api)
   # generate_api_param(doc, first_api, second_api, third_api)


 check if there is any duplicate incident found. 

rule 1:  check
check . 

 find duplicate incidents using machine-learning. 

rule 1:  find
rule 2.4:  find .  incident
second_api_param duplicate.
find . incident

 check if the candidate for duplicate incident score is above the threshold. 

rule 1:  check
rule 2.5:  check .  candidate
second_api_param 
check . candidate

 checks if incident has 'email/from' label 

rule 1:  check
rule 2.5:  check .  incident
second_api_param 
rule 3.5:  check .  incident .  email
third_api_param 
check . incident . email

 parse email with splunk 

rule 1:  email
rule 2.2:  parse . email
first_api_param   second_api_param  parse.
rule 3.1:  parse .  email . splunk
third_api_param 
parse . email . splunk

 parse email with nexpose 

rule 1:  email
rule 2.2:  parse . email
first_api_param   second_api_param  parse.
rule 3.1:  parse .  email . nexpose
third_api_param 
parse . email . nexpose

 parse email with sentinelone 

rule 1:  email
rule 2.2:  pa

In [71]:
spacy.explain('advcl')

'adverbial clause modifier'

In [10]:
def generate_api_param(doc, first_api, second_api, third_api):
    for token in doc:
        token_dep = token.dep_
        token_pos = token.pos_
        list_dep = ['compound', 'pobj', root, 'nsubj', 'dobj']
        if token_dep == compound and token_pos == noun:
            if token.head.pos_ in [verb, noun] and token.head.dep_ is root:
                param = token.head.lemma_
                third_api = token.lemma_
                print('rule 5: ', third_api, '(', param, ')')  
        elif token_dep == 'amod' and token_pos == 'ADJ':
            if token.head.dep_ == 'dobj' or 'pboj' and token.head.pos_ == 'noun':
                print('rule 6: ',token.lemma_, '.', token.head.lemma_)
        elif token_dep == 'appos' and token_pos == 'PROP':
            if token.head.dep_ == 'nsubjpass' and token.head.pos_ == noun:
                 print('rule 6.1: ', token.lemma_)
        elif token_dep == direct_obj and token_pos == noun:
            if token.head.dep_ == 'acl' or 'ccomp' and token.head.pos_== verb:
                if token.head.lemma_ == first_api:
                    first_api_param = token
                    print('rule 11: ',first_api, ' (', first_api_param, ')')
                if token.head.lemma_ == second_api:
                    second_api_param = token
                    print('rule 11: ', second_api, ' (', second_api_param, ')')
                if token.head.lemma_ == third_api:
                    third_api_param = token
                    print('rule 11: ', third_api, ' (', third_api_param, ')')
        elif token_dep == prep and token.pos_ == adp:
            if token.head.dep_ == 'appos' and token.head.pos_ == 'PRON':
                third_api = token.head.lemma_
                third_api_child = token.child.lemma_
                print('rule 3 ', third_api, third_api_child)
        elif token_dep == 'acl' or 'advcl' and token_pos == verb or noun:
            if token.head.dep_ == root and token.head.pos_==verb:
                #if len(child) > 0:
                   # parameter_api = token.lemma_
                        #print('rule 10: ', parameter_api)
                if token.head.dep_ == 'pobj' and token.head.pos_ == noun:
                    third_api = token.head.lemma_
                    third_api_param = token.children
                        #print('Rule 10 ', third_api, third_api_param)
        elif token_dep == compound and token_pos == noun:
            if token.head.pos_ in verb_noun and token.head.dep_ in list_dep:
                    # api_feature = token.lemma_
                    #param = token.head.lemma_
                third_api_head = token.lemma_
                third_api = token.head.lemma_
                print('rule 3.1: ', third_api)
        elif token_dep == compound and token_pos == noun:
            if token.head.pos_ in verb_noun and token.head.dep_ in list_dep:
                api_feature = token.lemma_
                param = token.head.lemma_
                # third_api_head = token.lemma_
                # third_api = token.head.lemma_
               
                #if token.head.dep_ == nominal_subj:
                    # third_api = token.head.lemma_
                    # print('rule 3.1: ', first_api, '. ', second_api, '.', third_api)
                #elif token.head.dep_ == direct_obj:
                    # second_api = token.head.lemma_
                    # print('rule 3.1: ', first_api, '. ', second_api, '.', third_api)
                
                # print('rule 3.1: ', first_api, '. ', second_api, '.', third_api)
         # elif token.head.dep_ == 'acl' or 'ccomp' and token.head.pos_==verb:
                    #  first_api = token.head.lemma_ 
               #if (token1.dep_==direct_obj or token.dep_ == 'nsubjpass') and token1.pos_== noun:
                            #second_api = token1.lemma_
                           # print('rule 3.1: ', first_api, '. ', second_api, '.', third_api)

In [11]:
count = 0
for sentence in sentence_list:
    if(count == 1):
        break
    count = count + 1
    print('\n', sentence, '\n')
    doc = nlp(sentence)
    first_api = second_api = third_api = first_api_feature = second_api_feature = third_api_feature = ""
    first_api, second_api, third_api = generate_api(doc)
    # generate_api_with_compound(doc)
    generate_api_param(doc, first_api, second_api, third_api)


 enrich accounts using one or more integrations 

rule 1:  enrich
rule 2.4:  enrich .  account
second_api_param 
rule 3.5:  enrich .  account .  integration
third_api_param 
